# 1. Loading Tags to RDS

#### Uncomment the following cell to download the data

In [1]:
import boto3
import warnings
import numpy as np
from Loaders import SQLLoader
warnings.filterwarnings("ignore")

In [2]:
ssm = boto3.client('ssm')
dbhost = ssm.get_parameter(Name='/moviestream/dbhost', WithDecryption=True)['Parameter']['Value']
dbuser = ssm.get_parameter(Name='/moviestream/dbuser', WithDecryption=True)['Parameter']['Value']
dbpass = ssm.get_parameter(Name='/moviestream/dbpass', WithDecryption=True)['Parameter']['Value']
dbname = ssm.get_parameter(Name='/moviestream/dbname', WithDecryption=True)['Parameter']['Value']

# A. Creating Loaders

In [3]:
file     = 'app-data/tags.csv'
dbengine = 'postgresql'
dbtable  = 'tags'
dtype    = {"userId":np.int64, "movieId":np.int64, "tag":np.str, "timestamp":np.int64}

tloader = SQLLoader(file, dbengine, dbhost, dbuser, dbpass, dbname, dbtable, drop=True, dtype=dtype)
display(tloader.df.head(3), tloader.status())

,userId,movieId,tag,timestamp,_insert,_insert_time,_update,_update_time,_delete,_delete_time
0,3,260,classic,1439472355,0,0,0,0,0,0
1,3,260,sci-fi,1439472256,0,0,0,0,0,0
2,4,1732,dark comedy,1573943598,0,0,0,0,0,0


{'state': 'AVAILABLE',
 'iteration': 0,
 'inserted': 0,
 'updated': 0,
 'deleted': 0,
 'indb': None}

--------

# B. SQL Loading
### B1. Full-Load

In [4]:
tloader.iud(inserts=100, updates=0 , deletes=0, max_registers=2000);

·[AVAILABLE] DB:2000 | I:2000, U:0, D:0 | I/s:856, U/s:0, D/s:0 | 95.00%


### B2. CDC (Inserts, Updates, Deletes)

In [7]:
tloader.iudx(inserts=100, updates=10, deletes=1)

# C. Kinesis Loading